## Scraper

Scrape webpages for DoB and DoD info.

Write compiled information to CSV

In [169]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
import altair as alt


## Make list of agent names

In [137]:
nameList=["Lord Kelvin", "Albert Einstein"]

## Scrape wikipedia

### Useful Functions

In [147]:
### get wikipedia info. box as text
def GetXmlInfo(inStr):

    nameStr="_".join(inStr.split(' '))
    url = f'http://en.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&rvsection=0&titles={nameStr}&format=xml'

    # print(url)

    res = requests.get(url)
    soup = BeautifulSoup(res.text, "xml")

    try:
        return soup.query.pages.revisions.getText()
    except Exception as e:
        print("Retieve xml info went wrong")
        print(f"An exception of type {type(e).__name__} occurred. Details: {str(e)}")
        return None

In [166]:
### get birth and death dates from info. text
def GetLifeSpan(xmlInfo):
    birth_data, death_data= None, None
    print("here")
    try:
        birth_re = re.search(r'(birth date(.*?)}})', xmlInfo)
        birth_data = birth_re.group(0).split('|')
    except AttributeError:
        birth_re = re.search(r'(Birth date(.*?)}})', xmlInfo)
        birth_data = birth_re.group(0).split('|')
    except Exception as e:
        print("Birth search went wrong")
        print(f"An exception of type {type(e).__name__} occurred. Details: {str(e)}")
    
    if birth_data!=None:
        birth_data=[bd.replace('{','').replace('}','') for bd in birth_data if "df" not in bd]
        birthStr=f"{birth_data[1]}-{birth_data[2]}-{birth_data[3]}"
        print("Got Birth info:",birthStr)
    else:
        birthStr=None

    try:
        death_re = re.search(r'(death date(.*?)}})', xmlInfo)
        death_data = death_re.group(0).split('|')
        print("there")
    except AttributeError:
        death_re = re.search(r'(Death date(.*?)}})', xmlInfo)
        death_data = death_re.group(0).split('|')
        print("or there")
    except Exception as e:
        print("Death search went wrong")
        print(f"An exception of type {type(e).__name__} occurred. Details: {str(e)}")

    if death_data!=None:
        death_data=[dd.replace('{','').replace('}','') for dd in death_data if "df" not in dd]
        deathStr=f"{death_data[1]}-{death_data[2]}-{death_data[3]}"
        print("Got Death info:",deathStr)
    else:
        deathStr=None

    return birthStr, deathStr


### do Scraping

In [167]:

data=[]
for name in nameList:
    print(f"### {name}")
    
    data.append({'name':name})

    xmlInfo=GetXmlInfo(name)
    # print(xmlInfo)

    if xmlInfo==None:
        print("No xmlInfo returned. skipping")
        continue

    # print(xmlInfo)
    dob, dod = GetLifeSpan(xmlInfo)
    
    data[-1]['born']=dob
    data[-1]['died']=dod

    

### Lord Kelvin
here
Got Birth info: 1824-06-26
there
Got Death info: 1907-12-17
### sdf sdfsd
Retieve xml info went wrong
An exception of type AttributeError occurred. Details: 'NoneType' object has no attribute 'getText'
No xmlInfo returned. skipping
### Albert Einstein
here
Got Birth info: 1879-3-14
or there
Got Death info: 1955-4-18


## Visualisation

### Table of data

In [168]:
df_data=pd.DataFrame(data)
df_data

,name,born,died
0,Lord Kelvin,1824-06-26,1907-12-17
1,sdf sdfsd,NaN,NaN
2,Albert Einstein,1879-3-14,1955-4-18


### Timeline chart

In [170]:
alt.Chart(df_data).mark_bar().encode(
    x='born:T',
    x2='died:T',
    y='name'
)

alt.Chart(...)